<a href="https://colab.research.google.com/github/hanhanwu/Hanhan_COLAB_Experiemnts/blob/master/try_federated_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nest_asyncio
nest_asyncio.apply()

%load_ext tensorboard

In [21]:
import collections
import numpy as np
import random
import pandas as pd
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_federated as tff

np.random.seed(10)

tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

In [12]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

16384/13273 [=====================================] - 0s 0us/step


In [14]:
df = pd.read_csv(csv_file)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [17]:
y = df['target']
X = df.drop('target', axis=1)

print(X.shape, y.shape)

(303, 13) (303,)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.1, 
                                                    random_state=10)

In [19]:

def create_clients(X, y, num_clients=10, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - feature set and the label.
        args: 
            X: feature set
            y: label
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(X, y))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [22]:
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [23]:
clients

{'client_1': [('fbs', 0)],
 'client_10': [('trestbps', 1)],
 'client_2': [('cp', 0)],
 'client_3': [('exang', 1)],
 'client_4': [('slope', 0)],
 'client_5': [('restecg', 0)],
 'client_6': [('thalach', 0)],
 'client_7': [('ca', 0)],
 'client_8': [('thal', 0)],
 'client_9': [('sex', 0)]}